### Notebook to parse text files to produce cleaned text of RPD (Legacy) decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Protection Division (Legacy) Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rpd/>.

Notes:

(1) Data Source: In the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). Because the IRB no longer regularly publishes RPD decisions, the dataset is no longer being updated, which is why we refer to the 
dataset as a legacy dataset. For more recent RPD decisions (obtained via Access to Information Requests), 
see the RLLR dataset.

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores who coded the initial parsing scripts for the Refugee Appeal Division Bulk Decisions Dataset, which were modified for this datset.


# Installing Libraries

In [ ]:
#!pip install langdetect
#!pip install regex
#!pip install dask

# Importing Libraries

In [1]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect, DetectorFactory
from difflib import get_close_matches
import json
from tqdm import tqdm
import chardet
import dask.bag as db
from dask.diagnostics import ProgressBar
import pathlib
from pathlib import Path
from datetime import datetime, timezone

## Declaring Constant
Here, we specify the directories containing our data files.

In [2]:
DATA_DIRS = ["../IRB Decisions - Initial Request - TEXT"]

# For SR:
DATA_DIRS = ["d:/IRB Decisions - Initial Request - TEXT/"]

In [3]:
# set seed for langdetect for consistent results and reproducibility
DetectorFactory.seed = 42



## Language Detection
This function determines the language of a given text.

In [4]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None


## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [5]:
def original_match_date_patterns(content):
    patterns = {
        "custom": (r"Date (?:of decision|de la décision)\s*\n\s*([A-Za-z]+)\s+(\d{1,2})\.\s*(\d{4})", lambda m: [m.group(1), m.group(2), m.group(3)]),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split()),
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)
    return None

def match_date_patterns(content):
    # 1. Try your proven patterns first
    result = original_match_date_patterns(content)
    if result:
        return result

    # 2. Fallback: Check lines after the label
    lines = content.splitlines()
    label_regex = re.compile(
        r'^\s*Date (of decision|de la décision)(\s*and reasons|\s*et des motifs)?\s*$', re.IGNORECASE)

    date_regexes = [
        # French style: 1er mai 2015, 24 juillet 2018, Le 24 juillet 2018
        re.compile(r'^(Le\s+)?(\d{1,2}|1er)\s+\w+\s+\d{4}$', re.IGNORECASE),
        # English style: January 21, 2019 or November 5th, 2014
        re.compile(r'^[A-Za-z]+\s+\d{1,2}(st|nd|rd|th)?\,?\s+\d{4}$'),
        # Dashes (rare but sometimes): 10-12-2018
        re.compile(r'^\d{1,2}-\d{1,2}-\d{4}$'),
    ]

    for idx, line in enumerate(lines):
        if label_regex.match(line):
            # Look ahead at the next 3 non-empty lines
            lookahead = 0
            for j in range(idx+1, min(idx+5, len(lines))):
                candidate = lines[j].strip()
                if not candidate:
                    continue
                lookahead += 1
                for reg in date_regexes:
                    m = reg.match(candidate)
                    if m:
                        cleaned = candidate.replace("Le ", "").replace(",", "")
                        return cleaned.split()
                if lookahead >= 3:
                    break
    
    return None

## Date Formatter
Takes detected regular expression and converts into one common format

In [6]:
french_to_english = {
        'janvier': 'January', 'fevrier': 'February', 'mars': 'March', 'avril': 'April',
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'aout': 'August',
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'decembre': 'December'
}

def correct_month_name(misspelled_month, possibilities=['Janvier','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    if correct_months:
        corrected_month = correct_months[0]
        # Check if the corrected month is in the French to English mapping
        return french_to_english.get(corrected_month.lower(), corrected_month)
    else:
        return misspelled_month

def correct_year_typo(year):
    if len(year) == 3 and year.startswith("0"):
        return "20" + year[1:]
    return year

def correct_year_typo(year):
    """Corrects year format typos."""
    return "20" + year[1:] if len(year) == 3 and year.startswith("0") else year

def process_numeric_format(parts):
    """Processes numeric date format 'dd-mm-yyyy'."""
    day, month, year = parts[0].split('-')
    year = correct_year_typo(year)
    return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

def process_day_first_format(parts, french_month_mapping):
    """Processes dates in 'day month year' format, French or English."""
    day = 1 if parts[0].lower() == '1er' else int(parts[0])

    month = ''
    # Check if month and year are concatenated
    if len(parts) == 2 and not parts[1].isdigit():
        month_year_str = parts[1]
        for i in range(1, len(month_year_str)):
            if month_year_str[i:].isdigit():
                month_str, year_str = month_year_str[:i], month_year_str[i:]
                year = correct_year_typo(year_str)
                month = french_month_mapping.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str.capitalize())
                break
    else:
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        if len(parts) >= 3:
            year = correct_year_typo(parts[2])

    if month in french_month_mapping:
        return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
    else:
        if isinstance(month, int):
            return datetime(int(year), month, day).date().strftime('%Y-%m-%d')
        
        corrected_month = correct_month_name(month.capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

def process_month_first_format(parts):
    """Processes month first format with possible ordinal suffix."""
    day = 0
    month = ''
    year = ''
    
    if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
        
        if parts[1].isdigit() and len(parts[1]) > 4: 
            month = parts[0]
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = year_str
            day = int(day_str)
            
        elif parts[1].isdigit()and len(parts[1]) > 3: #Year is the second entry
            month_day_str = parts[0]
            for i in range(1, len(month_day_str)):
                if not month_day_str[i].isdigit():
                    day_str, month_str = month_day_str[:i], month_day_str[i:]
                    if day_str.isdigit():
                        day = int(day_str)
                    month = french_to_english.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str)
                    parts[0] = month
                    break
            year = parts[1]
        else:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)

    else:
        day = re.sub(r"[^\d]", "", parts[1])
        day = int(day) if day.isdigit() else 1
        if len(parts) >= 3:
            year = correct_year_typo(parts[2])
            
    try:
        corrected_month = correct_month_name(parts[0].capitalize())
        return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [7]:
def process_date_parts(parts, french_month_mapping):
    """Determines the correct date processing method based on the format of the parts."""
    if '-' in parts[0]:
        return process_numeric_format(parts)
    elif parts[0].isdigit() or parts[0].lower() == '1er':
        return process_day_first_format(parts, french_month_mapping)
    else:
        return process_month_first_format(parts)

def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [8]:
def extract_rpd_number(content):
    """Extracts the RPD number from the content, ignoring IAD/ID/RAD files."""
    # Check for lines indicating the file should be ignored
    ignore_lines = ["IAD File",
                    "dossier de la SAI",
                    "IMMIGRATION APPEAL DIVISION", 
                    "ID File", 
                    "IMMIGRATION DIVISION", 
                    "RAD File",
                    "REFUGEE APPEAL DIVISION",
                    "REFUGEE DIVISION"
                    ]
    
    for line in content.splitlines():
        # Sanitize the current line
        sanitized_line = ''.join(c for c in line if c.isprintable()).strip()

        # Check if the sanitized line matches any ignore line
        if any(ignore_line in sanitized_line for ignore_line in ignore_lines):   
            return None
        
        if "RPD File" in sanitized_line or "RPD file" in sanitized_line :          
            rpd_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rpd_number_match:
                return rpd_number_match.group(1)
            
            # If RPD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rpd_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rpd_number_match:
                    return rpd_number_match.group(1)

    if "REFUGEE PROTECTION DIVISION" in content or "SECTION DE LA PROTECTION DES RÉFUGIÉS" in content:
        rpd_number_match = re.search(r"([A-Z]{2}\d+-\d+)", content)
        if rpd_number_match:
            return rpd_number_match.group(1)

    return None

def process_file(file_path):
    """Processes a single file and extracts data."""

    # manually exclude problematic files
    problem_files = ["1821419", "636351"]
    if Path(file_path).stem in problem_files:
        return None
    
        
    # Use chardet to detect the encoding of the file
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    encoding = chardet.detect(raw_data)['encoding']

    # Read the file with the detected encoding
    with open(file_path, 'r', errors='replace', encoding=encoding) as file:
        content = file.read()

    # Map problematic file fragments to their corresponding RPD numbers
    problem_files = {
            "1690585.txt": "TA9-19655",
            "1740368.txt": "VB4-01572",
            "1777359.txt": "MB1-02173",
            "3270676.txt": "MB8-15979",
            "486492.txt": "VA1-02547",
            "3271970.txt": "TB6-09748",
            "2999214.txt": "TB4-02413",
            "2222199.txt": "VB4-00719",
            "2243526.txt": "TB0-05693",
            "1813128.txt": "TB2-04162",
            "1783566.txt": "VB2-02093",
            "1545938.txt": "VB2-01056",
            "1108895.txt": "TA4-11055",
            "1111316.txt": "MA5-03423",
            "1849493.txt": "MB4-05069",
            "450084.txt": "TA0-10901",
            "575083.txt": "TA3-21649",
            "1105731.txt": "TA4-15299",
            "522031.txt": "TA4-16477",
            "1069897.txt": "TA5-02029",
            "605641.txt": "TA5-12899",
            "450696.txt": "VA3-04732",
            "442828.txt": "VA4-00076",
            "3170563.txt": "VB8-02364",
            "3270677.txt": "MB8-15979",
            "450617.txt": "TA0-05246",
            "486417.txt": "TA0-09408",
            "638214.txt": "TA1-08064",
            "486449.txt": "TA2-00027",
            "1069901.txt": "TA3-06172",
            "1108936.txt": "TA5-13980",
            "1175514.txt": "TA9-08309",
            

    }

    # Check for problem files using a loop, then extract RPD number if not found
    rpd_number = None
    for fragment, rpd in problem_files.items():
        if fragment in file_path:
            rpd_number = rpd
            break

    if rpd_number is None:
        rpd_number = extract_rpd_number(content)

    if rpd_number:
        lang = detect_language(content)
        document_date = extract_document_date(content)
        
        scraped_timestamp = datetime.fromtimestamp(os.path.getmtime(file_path), tz=timezone.utc)


        return {
            'citation': rpd_number,
            'citation2': '',
            'dataset': 'RPD',
            'name': '',
            'language': lang,
            'document_date': document_date,
            'url': os.path.basename(file_path),
            'scraped_timestamp': scraped_timestamp,
            'referrer': 'a2aj_parse_rpd_txt.ipynb',
            'unofficial_text': content,
            
        }
    return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe.

In [9]:
# Main data processing loop (run in paraellel using Dask)

def process_file_wrapper(file_path):
    if not os.path.basename(file_path).startswith('~'):
        return process_file(file_path)

# Gather all file paths
file_paths = []
for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        dir_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
        file_paths.extend(dir_files)

# Create a Dask Bag from file paths
file_bag = db.from_sequence(file_paths)

# Use Dask to process files in parallel
with ProgressBar():
    results = file_bag.map(process_file_wrapper).filter(lambda x: x is not None).compute()

# Convert results to a Pandas DataFrame
df = pd.DataFrame(results)

# # Process files using a regular loop for easier debugging if needed
# results = []
# for file_path in file_paths:
#     try:
#         result = process_file_wrapper(file_path)
#         if result is not None:
#             results.append(result)
#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")

# # Convert results to a Pandas DataFrame
# df = pd.DataFrame(results)

[########################################] | 100% Completed | 228.28 s


In [10]:
df_backup = df.copy()

In [11]:
df = df_backup.copy()

### Data cleaning
Cleans data to match huggingface dataset

In [12]:
### Initially removed these, not sure why, have not removed.
# "1025049.txt",
# "1529910.txt",
# "1742145.txt",
# "638230.txt",
# "1024036.txt",
# "1025063.txt",
# "1742152.txt",
# "547985.txt",
# "638231.txt",
# "1529916.txt",

# Manually fix some dates:
print ("Starting df", len(df))

# manually remove some files
problem_files = ["1096872.txt",  # Unclear why this and two next files aren't working properly
                 "1180624.txt",  # ditto
                 "1096923.txt",  # ditto
                 "1271563.txt",  # Again, something weird with this file and the next three
                 "1271558.txt",  # ditto
                 "1024046.txt",  # ditto
                 "1024036.txt",  # ditto
                 
                ]
df = df[~df['url'].isin(problem_files)].reset_index(drop=True)

# manually remove all rows where citation is "TB0-12466" ## This is a problematic file
df = df[df['citation'] != "TB0-12466"].reset_index(drop=True)

# manually fixe some languages
df.loc[df['url'] == '1095022.txt', 'language'] = 'fr'
df.loc[df['url'] == '3130684.txt', 'language'] = 'fr'


# use gen_ai to fix missing date
# import DATA/ai_date_dict.json
import json
with open("DATA/ai_date_dict.json", "r") as f:
    ai_date_dict = json.load(f)
for url, date in ai_date_dict.items():
    if url in df['url'].values:
        df.loc[df['url'] == url, 'document_date'] = date

# manually fix some document dates
df.loc[df['url'] == '1522298.txt', 'document_date'] = '2023-11-13' 
df.loc[df['url'] == '1547144.txt', 'document_date'] = '2006-09-29' 
df.loc[df['url'] == '1783693.txt', 'document_date'] = '2012-06-04' 
df.loc[df['url'] == '1813165.txt', 'document_date'] = '2013-09-17' 
df.loc[df['url'] == '955532.txt', 'document_date'] = '2008-11-20' 
df.loc[df['url'] == "1105710.txt", 'document_date'] = "2006-03-03"
df.loc[df['url'] == "1108993.txt", 'document_date'] = "2005-10-04"
df.loc[df['url'] == "1783566.txt", 'document_date'] = "2005-10-04"
df.loc[df['url'] == "1813268.txt", 'document_date'] = "2013-05-15"
df.loc[df['url'] == "442802.txt", 'document_date'] = "2006-03-31"
df.loc[df['url'] == "3207808.txt", 'document_date'] = "2018-12-07"
df.loc[df['url'] == "3207784.txt", 'document_date'] = "2019-05-28"
df.loc[df['url'] == "3061542.txt", 'document_date'] = "2018-11-05"
df.loc[df['url'] == "2931707.txt", 'document_date'] = "2018-01-23"
df.loc[df['url'] == "2999223.txt", 'document_date'] = "2018-05-14"
df.loc[df['url'] == "2897156.txt", 'document_date'] = "2018-03-07"
df.loc[df['url'] == "2830240.txt", 'document_date'] = "2017-09-21"
df.loc[df['url'] == "2979072.txt", 'document_date'] = "2017-09-15"
df.loc[df['url'] == "2229868.txt", 'document_date'] = "2016-06-07"
df.loc[df['url'] == "2059472.txt", 'document_date'] = "2016-02-04"
df.loc[df['url'] == "1783566.txt", 'document_date'] = "2013-10-11"
df.loc[df['url'] == "1513501.txt", 'document_date'] = "2013-11-13"
df.loc[df['url'] == "1813100.txt", 'document_date'] = "2013-08-01"
df.loc[df['url'] == "1813268.txt", 'document_date'] = "2013-05-31"
df.loc[df['url'] == "1111316.txt", 'document_date'] = "2007-05-03"
df.loc[df['url'] == "1024860.txt", 'document_date'] = "2007-10-09"

print("After removing problem dates and files:", len(df))

# fix dates format
df['document_date'] = pd.to_datetime(df['document_date'], errors='coerce', utc=True)
# Sort by document_date descending, so newest dates are first; NaT is last
df = df.sort_values(by=['document_date', 'scraped_timestamp'], ascending=[False, False])

# print number of rows with no document_date
print("Number of rows with no document_date:", df['document_date'].isna().sum())
#print(df[df['document_date'] < pd.Timestamp('2013-01-01', tz='UTC')]['url'].tolist())

# export to parquet the rows where there is no document_date
df_no_date = df[df['document_date'].isna()]
df_no_date.to_parquet("DATA/df_no_date.parquet", index=False)

# Remove rows where year document_date is before 2001 or after 2020
# print number of rows before 2002 (inclusive)
print("Number of rows before 2002:", df[df['document_date'].dt.year < 2002].shape[0])
# print number of rows after 2020 (inclusive)
print("Number of rows after 2020:", df[df['document_date'].dt.year > 2020].shape[0])
df = df[df["document_date"].dt.year.between(2002, 2020, inclusive="both")].reset_index(drop=True)
print ("After removing before 2002 and after 2020", len(df)) 


df



Starting df 13859
After removing problem dates and files: 13850
Number of rows with no document_date: 2
Number of rows before 2002: 0
Number of rows after 2020: 101
After removing before 2002 and after 2020 13747


,citation,citation2,dataset,name,language,document_date,url,scraped_timestamp,referrer,unofficial_text
0,MB7-15824,,RPD,,en,2020-12-14 00:00:00+00:00,3594183.txt,2023-11-13 17:19:52.957924+00:00,a2aj_parse_rpd_txt.ipynb,RPD File No. / No de dossier de la SPR : MB7-1...
1,MB7-15824,,RPD,,fr,2020-12-14 00:00:00+00:00,3594182.txt,2023-11-13 17:19:52.779925+00:00,a2aj_parse_rpd_txt.ipynb,Dossier de la SPR / RPD File: MB7-15824\nIUC /...
2,MB1-07314,,RPD,,en,2020-12-10 00:00:00+00:00,3590891.txt,2023-11-13 17:19:27.284038+00:00,a2aj_parse_rpd_txt.ipynb,RPD File / Dossier de la SPR: MB1-07314\nMB1-0...
3,MB1-07314,,RPD,,fr,2020-12-10 00:00:00+00:00,3590890.txt,2023-11-13 17:19:27.045998+00:00,a2aj_parse_rpd_txt.ipynb,Dossier de la SPR / RPD File: MB1-07314\...
4,MB7-07470,,RPD,,fr,2020-12-02 00:00:00+00:00,3590897.txt,2023-11-13 17:19:28.332317+00:00,a2aj_parse_rpd_txt.ipynb,Dossier de la SPR / RPD File: MB7-07470\nMB7-0...
...,...,...,...,...,...,...,...,...,...,...
13742,VA1-02181,,RPD,,en,2002-07-31 00:00:00+00:00,638280.txt,2023-11-13 17:42:33.250329+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\n\nRefugee Prote...
13743,MA1-05556,,RPD,,fr,2002-07-30 00:00:00+00:00,638112.txt,2023-11-13 17:42:08.174968+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration et du statut de ré...
13744,MA1-05556,,RPD,,en,2002-07-30 00:00:00+00:00,638111.txt,2023-11-13 17:42:08.044931+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration et du statut de r...
13745,VA2-01049,,RPD,,fr,2002-07-16 00:00:00+00:00,638203.txt,2023-11-13 17:42:21.773916+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\n\nRefugee Prote...


In [13]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # # Remove single newlines
    # text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

tqdm.pandas()
df['unofficial_text'] = df.unofficial_text.progress_apply(clean_text)

100%|██████████| 13747/13747 [00:18<00:00, 743.32it/s]


In [14]:
# get df_en where language is English
df_en = df[df['language'] == 'en'].reset_index(drop=True)
# get df_fr where language is French
df_fr = df[df['language'] == 'fr'].reset_index(drop=True)

# print how many df_en.citation is duplicated
print("Number of duplicated citations in df_en:", df_en.citation.duplicated().sum())
print("Number of duplicated citations in df_fr:", df_fr.citation.duplicated().sum())

# print how many rows in df_en are duplicated for BOTH citation and document_date
print("Number of duplicated rows in df_en for BOTH citation and document_date:", df_en.duplicated(subset=['citation', 'document_date']).sum())
print("Number of duplicated rows in df_fr for BOTH citation and document_date:", df_fr.duplicated(subset=['citation', 'document_date']).sum())

# export to parquet the rows where there are duplicated citations in df_en
df_en_duplicates = df_en[df_en.duplicated(subset=['citation'], keep=False)]
df_en_duplicates.to_parquet("DATA/df_en_duplicates.parquet", index=False)
# export to parquet the rows where there are duplicated citations in df_fr
df_fr_duplicates = df_fr[df_fr.duplicated(subset=['citation'], keep=False)]
df_fr_duplicates.to_parquet("DATA/df_fr_duplicates.parquet", index=False)   

# Blunt removal of duplicates in df_en and df_fr
df_en = df_en.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_en = df_en.drop_duplicates(subset=['unofficial_text'], keep='first').reset_index(drop=True)
df_en = df_en.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_en = df_en.drop_duplicates(subset=['citation'], keep='first').reset_index(drop=True)

df_fr = df_fr.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_fr = df_fr.drop_duplicates(subset=['unofficial_text'], keep='first').reset_index(drop=True)
df_fr = df_fr.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_fr = df_fr.drop_duplicates(subset=['citation'], keep='first').reset_index(drop=True)

# Print the number of rows after removing duplicates
print("Number of rows in df_en after removing all duplicate citations:", len(df_en))
print("Number of rows in df_fr after removing duplicate citations:", len(df_fr))

# number of rows in df_en that does not have an identical citation in df_fr
print(
    "Num rows in df_en with no citation match in df_fr",
    df_en[~df_en['citation'].isin(df_fr['citation'])].reset_index(drop=True).shape[0]
)
# number of rows in df_fr that does not have an identical citation in df_en
print(
    "Num rows in df_fr with no citation match in df_en",
    df_fr[~df_fr['citation'].isin(df_en['citation'])].reset_index(drop=True).shape[0]
)



# for df_en add "_en" to end of all cols
df_en.columns = [col + "_en" for col in df_en.columns]
# for df_fr add "_fr" to end of all cols
df_fr.columns = [col + "_fr" for col in df_fr.columns]

# merge df_en and df_fr on citation_en and citation_fr, keeping only matches
df = pd.merge(df_en, df_fr, left_on='citation_en', right_on='citation_fr', how='inner')

# rename dataset_en to dataset, and drop unused cols
df = df.rename(columns={'dataset_en': 'dataset'})
df = df.drop(columns=['dataset_fr', 'language_en', 'language_fr'])


# print len of final df
print("Number of rows in final df:", len(df))
print("*********** AT SOME POINT RECONSIDER THIS ***********")
df.head(5)

Number of duplicated citations in df_en: 127
Number of duplicated citations in df_fr: 126
Number of duplicated rows in df_en for BOTH citation and document_date: 14
Number of duplicated rows in df_fr for BOTH citation and document_date: 13
Number of rows in df_en after removing all duplicate citations: 6750
Number of rows in df_fr after removing duplicate citations: 6744
Num rows in df_en with no citation match in df_fr 21
Num rows in df_fr with no citation match in df_en 15
Number of rows in final df: 6729
*********** AT SOME POINT RECONSIDER THIS ***********


,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,referrer_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,referrer_fr,unofficial_text_fr
0,AA0-01604,,RPD,,2002-11-08 00:00:00+00:00,638019.txt,2023-11-13 17:41:54.418722+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...,AA0-01604,,,2002-11-08 00:00:00+00:00,638020.txt,2023-11-13 17:41:54.561741+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...
1,AA1-00073,,RPD,,2002-11-06 00:00:00+00:00,638027.txt,2023-11-13 17:41:55.585303+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...,AA1-00073,,,2002-11-06 00:00:00+00:00,638028.txt,2023-11-13 17:41:55.719812+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...
2,AA1-00163,,RPD,,2003-03-13 00:00:00+00:00,486650.txt,2023-11-13 17:26:04.330330+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...,AA1-00163,,,2003-03-13 00:00:00+00:00,486649.txt,2023-11-13 17:26:04.220815+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...
3,AA1-00454,,RPD,,2002-11-01 00:00:00+00:00,638041.txt,2023-11-13 17:41:57.785440+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...,AA1-00454,,,2002-11-01 00:00:00+00:00,638042.txt,2023-11-13 17:41:57.939957+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...
4,AA1-00611,,RPD,,2003-10-24 00:00:00+00:00,486305.txt,2023-11-13 17:25:27.926737+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...,AA1-00611,,,2003-10-24 00:00:00+00:00,486306.txt,2023-11-13 17:25:28.065251+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and Refugee Board\nRefugee Protect...


In [15]:
# how many rows in df have document_date_en and document_date_fr that are not the same
print("Number of rows in df with different document_date_en and document_date_fr:", 
      df[df['document_date_en'] != df['document_date_fr']].shape[0])


Number of rows in df with different document_date_en and document_date_fr: 0


In [16]:
df[df['document_date_en'] != df['document_date_fr']]



,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,referrer_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,referrer_fr,unofficial_text_fr


In [17]:
# are there any duplicate citations_en in df?
print("Number of duplicated citations_en in df:", df.citation_en.duplicated().sum())
# are there any duplicate citations_fr in df?
print("Number of duplicated citations_fr in df:", df.citation_fr.duplicated().sum())

# are there any blank or null or nan citations_en in df?
print("Number of blank or null or nan citations_en in df:", df['citation_en'].isnull().sum())
# are there any blank or null or nan citations_fr in df?
print("Number of blank or null or nan citations_fr in df:", df['citation_fr'].isnull().sum())

Number of duplicated citations_en in df: 0
Number of duplicated citations_fr in df: 0
Number of blank or null or nan citations_en in df: 0
Number of blank or null or nan citations_fr in df: 0


### Exports

In [19]:
# export cleaned df to parquet
out_path_parsed = pathlib.Path('DATA/rpd_cases.parquet')
df.to_parquet(out_path_parsed, index=False)

In [18]:
# export to Mongo # NEED TAILSCALE ON
from pymongo import MongoClient
from datetime import timezone
import numpy as np

MONGO_URL = 'mongodb://rll-tr7970x:27017/'
MONGO_DB = 'case-scraping'
MONGO_COLLECTION = 'rpd-scrapes'
client = MongoClient(MONGO_URL, tz_aware=True, tzinfo=timezone.utc)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

# drop existing collection if it exists
if collection.count_documents({}) > 0:
    collection.drop()

def strip_nulls(recs):
    return [{k: v for k, v in rec.items() if v is not None} for rec in recs]

def sanitize_records(recs):
    """Return a NEW list of dicts that PyMongo can store."""
    return [
        {k: _clean_scalar(v) for k, v in rec.items()}
        for rec in recs
    ]

def _clean_scalar(v):
    # 1) All “missing” markers: <NA>, NaN, NaT  →  None
    if pd.isna(v):
        return None
    
    # 2) pandas Timestamp  →  native datetime with UTC tzinfo
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime().astimezone(timezone.utc)
    
    # 3) NumPy scalar (np.int64, np.float64, …)  →  Python int/float
    if isinstance(v, np.generic):
        return v.item()
    
    return v

records = df.to_dict(orient="records")
clean_docs = strip_nulls(sanitize_records(records))
result = collection.insert_many(clean_docs)


TOOLS


In [ ]:
# load .env data
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("azure_api")
azure_url = os.getenv("azure_url")

# setup Azure OpenAI client
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = azure_url + "gpt-4.1-mini"
model_name = "gpt-4.1-mini"
system_message = "You are an assistant to a Canadian refugee law professor."

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key),    
)

def get_response(prompt, system_message=system_message, model_name=model_name, max_tokens = 20):
    response = client.complete(
        messages=[
            SystemMessage(content=system_message),
            UserMessage(content=prompt),
        ],
        max_tokens=max_tokens,
        temperature=0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model=model_name
    )
    return response.choices[0].message.content

# load "DATA/df_no_date.parquet"
import pandas as pd
df_no_date = pd.read_parquet("DATA/df_no_date.parquet")

system_message = """
You are an assistant to a Canadian refugee law professor.
You are given the text of an RPD decision. Your task is to extract
the date of the decision in the string format 'YYYY-MM-DD'.
If you are not sure, return the string 'Unknown'.
Do not return any other text.
"""

for index, row in df_no_date.iterrows():
    print(f"Processing row {index+1}/{len(df_no_date)}")
    text = row['unofficial_text']
    prompt = f"Extract the date of the decision from the following text\n\nDECISON:\n\n{text}\n\nDATE OF DECISION IN YYYY-MM-DD FORMAT ('Unknown' if unsure):"
    response = get_response(prompt, system_message=system_message, model_name=model_name, max_tokens=20)
    df_no_date.at[index, 'ai_document_date'] = response.strip()

# export df_no_date to parquet, with name ai_gen_df_no_date.parquet

df_no_date.to_parquet("DATA/ai_gen_df_no_date.parquet", index=False)

# drop rows where ai_document_date is 'Unknown'
df_no_date = df_no_date[df_no_date['ai_document_date'] != 'Unknown'].reset_index(drop=True) 

# create a dictionary of url: ai_document_date
ai_date_dict = df_no_date.set_index('url')['ai_document_date'].to_dict()

# export ai_date_dict to json
import json
with open("DATA/ai_date_dict.json", "w") as f:
    json.dump(ai_date_dict, f, indent=4)

df_no_date


In [ ]:
# Create a Dask Bag from file paths
file_bag = db.from_sequence(file_paths)

def get_text(file_path):
    """Read the file and return its content."""
    with open(file_path, 'r', errors='replace', encoding='utf-8') as file:
        file_name = os.path.basename(file_path)
        return {"file": file_name, "text": file.read()}

# Use Dask to process files in parallel
with ProgressBar():
    results = file_bag.map(get_text).filter(lambda x: x is not None).compute()

# Convert results to a Pandas DataFrame
df_all = pd.DataFrame(results)

In [ ]:
# check if "SPR File" is in df_all.text
df_all['has_spr_file'] = df_all['text'].str.contains("SPR File", na=False)

# len of df_all where has_sar_file is True
print("Number of rows in df_all with has_spr_file True:", df_all[df_all['has_spr_file']].shape[0])

df_all[df_all['has_spr_file']].head(20)

In [ ]:
import difflib
fr_list=df_fr[~df_fr['url_fr'].isin(df['url_fr'])].reset_index(drop=True)['citation_fr'].to_list()
en_list=df_en[~df_en['url_en'].isin(df['url_en'])].reset_index(drop=True)['citation_en'].to_list()
matches = []
cutoff = 0.8
for en_code in en_list:
    close = difflib.get_close_matches(en_code, fr_list, n=1, cutoff=cutoff)
    if close:
        ratio = difflib.SequenceMatcher(None, en_code, close[0]).ratio()
        matches.append((en_code, close[0], round(ratio, 2)))
df_matches = pd.DataFrame(matches, columns=['citation_en', 'citation_fr_candidate', 'similarity'])
df_matches = df_matches.sort_values('similarity', ascending=False)

# Print or export

print(df_matches)

In [ ]:
citation_sought = "VB0-04055"
df_backup[df_backup['citation'] == citation_sought]

In [ ]:
# show rows in df_all where citation is citation_sought
df_all[df_all['text'].str.contains(citation_sought, na=False)].head(10)

In [ ]:
# checked these:
# AA4-000384
# MA7-08092
# MB7-01793	
# MB9-01194
# TA6-15248
# TA7-14813	
# TB0-06418	
# TB0-12466	
# TB1-07022
# TB2-04162
# TB2-14479
# TB3-02309
# TB3-14343
# TB8-07128
# VB0-04055


df_fr[~df_fr['url_fr'].isin(df['url_fr'])].reset_index(drop=True).tail(10)

In [ ]:
# checked these:
# MA9-09154
# TA5-02094
# TA5-02334
# TA5-04195
# TA5-07065	
# TA7-14226	
# TA8-06636	
# TB0-11041	
# TB0-14316	
# TB2-01830	
# TB2-02214
# TB3-08302
# TB4-00686	
# VA3-03919	
# VA8-03935	
# VB0-00648	
# VB0-03563	
# VB2-00249	

df_en[~df_en['url_en'].isin(df['url_en'])].reset_index(drop=True).tail(10)
#df_fr[~df_fr['url_fr'].isin(df['url_fr'])].reset_index(drop=True)

In [ ]:
# # To compare duplicates
# import difflib

# # Get the two texts
# texts = df_fr[df_fr.duplicated(subset=['citation', 'document_date'], keep=False)].unofficial_text.iloc[[0,1]]

# # Compute the diff
# diff = difflib.unified_diff(
#     texts.iloc[0].splitlines(),
#     texts.iloc[1].splitlines(),
#     fromfile='text0',
#     tofile='text1',
#     lineterm=''
# )

# # Print the diff
# for line in diff:
#     print(line)